# Building the data sets

We will here build the train, target and test set.
We will store the datasets so they can nbe easily accessed when training and making predictions

# TOC

* [Loading the data](#Loading-the-data)
* [Cleaning of the dataset](#Cleaning-of-the-dataset)
* [Adding date features](#Adding-date-features)
* [Adding shop features](#Adding-shop-features)
* [Adding item features](#Adding-item-features)
* [Adding text features](#Adding-text-features)
* [Adding leakage features](#Adding-leakage-features)
* [Adding mean encoding](#Adding-mean-encoding)
* [Feature dropping](#Feature-dropping)
* [Normalization](#Normalization)
* [Checking correlation](#Checking-correlation)
* [Adding temporal history](#Adding-temporal-history)
* [Storing the data frames](#Storing-the-data-frames)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
import requests
import re
import nltk
import sklearn
import gc
import pandas as pd
import numpy as np
from pathlib import Path
from itertools import product
from dateutil.relativedelta import relativedelta

We will here generate the following features

* Date features
    * Year number
    * Month number
    * Days in month
    * Quarter
    * Holidays the current month
    * Holidays the previous month
    * Holidays the next month
* Shop features 
    * Shop total revenue current month
    * Shop total items sold current month
    * Shop number of products
* Item features 
    * Sum of item count aggregated by shop, item and month (target)
    * Max of item count aggregated by shop, item and month 
    * Min of item count aggregated by shop, item and month 
    * Mean of item count aggregated by shop, item and month 
    * Sum of item count aggregated by item and month 
    * Max of item count aggregated by item and month 
    * Min of item count aggregated by item and month 
    * Mean of item count aggregated by item and month 
* Text features
    * TF-IDF of item names, item category names and shops names
    * Word count of cyrillic words in item names, item category names and shops names
    * Word count of latin words in item names, item category names and shops names
    * Total word count of words in item names, item category names and shops names
* Leakage features
    * Item occurences in the test set
    * ID
    * Row number
* Mean encoding
    * Expanding mean using the item id

**NOTE**: As not all items are present in the training set we must be careful when we are filling out the missing values in the test set

Other features which could have been generated

* Geographical features of the shops
    * Shop located in big city
    * Nearest geographical neighbors

# Loading the data

In [ ]:
data_dir = Path('.').absolute().joinpath('data')

sales_train = pd.read_csv(data_dir.joinpath('sales_train.csv.gz'))
sales_test = pd.read_csv(data_dir.joinpath('test.csv.gz'))
items = pd.read_csv(data_dir.joinpath('items.csv'))
item_categories = pd.read_csv(data_dir.joinpath('item_categories.csv'))
shops = pd.read_csv(data_dir.joinpath('shops.csv'))

In [ ]:
n_train_samples = sales_train.shape[0]
n_test_samples = sales_test.shape[0]

Cast the dates to actual dates for easier manipulation

In [ ]:
sales_train.loc[:, 'date'] = pd.to_datetime(sales_train.loc[:, 'date'], format='%d.%m.%Y')

# Cleaning of the dataset

After investigating the item count per day outliners we saw that these may actual be correct (and not arising from typos etc.)

Further, we saw that the outliner in price could be fixed, by converting `'Radmin 3  - 522 лиц.'` to `'Radmin 3  - 1 лиц.'`.

We do the conversion in the following cells

In [ ]:
# Values obtained from EDA
item_count_522 = 522
item_id_1 = 6065

In [ ]:
max_price = sales_train.loc[:, 'item_price'].max()
high_price = sales_train.loc[sales_train.loc[:, 'item_price'] == max_price]

In [ ]:
index = high_price.index[0]

sales_train.loc[index, 'item_id'] = item_id_1
sales_train.loc[index, 'item_cnt_day'] = item_count_522
sales_train.loc[index, 'item_price'] = max_price/item_count_522

Further, we saw that the datapoint for plastic bags had a high item count. As this is used to calculate the renevue below, we will not alter this item

We've seen that that `item_cnt_day` are of floats, to speed up calculations, we transform them to integers

In [ ]:
sales_train.loc[:, 'item_cnt_day'] = sales_train.loc[:, 'item_cnt_day'].astype(np.int32)

# Making the base set

We will here use the trick from the ensembling exercise where we create an outer product of the `item_id` and `shop_id` present in each block.

Recall that we are predicting for the `34`th month

In [ ]:
sales_test.loc[:, 'date_block_num'] = 34

In [ ]:
data = []
cols = ['date_block_num','shop_id','item_id']

# The training part
for block in sales_train.loc[:,'date_block_num'].unique():
    tmp = sales_train.loc[sales_train.loc[:, 'date_block_num'] == block, cols]
    # NOTE: Here we make an outer product of 'date_block_num','shop_id' and 'item_id'
    data.append(np.array(list(product([block], tmp.loc[:,'shop_id'].unique(), tmp.loc[:,'item_id'].unique()))))
    
# Make a sorted dataset of the list
data = pd.DataFrame(np.vstack(data), columns=cols).sort_values(cols)

We do not want to expand the test set, as we will use this for predictions

In [ ]:
data = pd.concat([data, sales_test.loc[:, ['date_block_num','shop_id','item_id']]], axis=0)

# Adding date features

In [ ]:
dates = sales_train.loc[:, ['date_block_num', 'date']]

# Add one date for the predict date
# NOTE: The relativedelta module takes care of problems with dates ending with 28, 30, 31
next_month = dates.loc[:, 'date'].max() + relativedelta(months=1)
next_date_block_num = dates.loc[:, 'date_block_num'].max() + 1
test_month = pd.DataFrame({'date_block_num':[next_date_block_num], 'date':[next_month]})
dates = pd.concat([dates, test_month], axis=0)

In [ ]:
dates['date'].max()

Recall from the EDA that we found out that the last date in dataset was `2015-10-31`, this means we are going to predict for `2015-11`. 

Further, we note that only the year and month data is present in the test dataset, meaning that using information on the day level does not make sense.

### Standard date features

We here add date features as seasonal trends are present in the dataset

In [ ]:
dates.loc[:, 'year'] = dates.loc[:, 'date'].dt.year
dates.loc[:, 'month'] = dates.loc[:, 'date'].dt.month
dates.loc[:, 'days_in_month'] = dates.loc[:, 'date'].dt.days_in_month
dates.loc[:, 'quarter'] = dates.loc[:, 'date'].dt.quarter

### Holidays

We will here generate the number of holidays in the previous month, the current month and the next month

In [ ]:
def get_russian_holidays(year):
    """
    Returns a Series of Russian holidays in a given year
    
    Parameters
    ----------
    year : int
        The year to investigate
    
    Returns
    -------
    holidays : Series
        Series of the holidays on datetime64 format
    """
    
    url = f'https://www.timeanddate.com/holidays/russia/{year}'
    html = requests.get(url).content
    # A list is returned
    table_df = pd.read_html(html)[0]
    # Rename
    table_df = table_df.rename(columns={'Date': 'date'})
    holidays = pd.to_datetime(table_df['date'], format='%b %d')
    
    # Replace the year and cast to datetime
    holidays = holidays.apply(lambda x: x.replace(year=year))

    return holidays

In [ ]:
def get_year_months_len(df):
    """
    Returns the number of entries grouped by year and month of the input data frame
    
    Parameters
    ----------
    df : DataFrame
        DataFrame with a column named 'date'
    
    Returns
    -------
    df : DataFrame
        The input DataFrame where the number of entries grouped by year and month
        is appended to the column named 'year_month_count' 
    """
    
    new_df = df.copy()
    
    new_df.loc[:, 'year'] = new_df.loc[:, 'date'].dt.year
    new_df.loc[:, 'month'] = new_df.loc[:, 'date'].dt.month
    
    df.loc[:, 'year_month_count'] = new_df.groupby(['year', 'month'])['date'].transform(len)
    
    return df

In [ ]:
# NOTE: We include 2012 to get the first prev_holiday_count later
holiday_2012 = get_russian_holidays(2012).to_frame()
holiday_2013 = get_russian_holidays(2013).to_frame()
holiday_2014 = get_russian_holidays(2014).to_frame()
holiday_2015 = get_russian_holidays(2015).to_frame()
holidays = pd.concat([holiday_2012, holiday_2013, holiday_2014, holiday_2015])

In [ ]:
holiday_count = get_year_months_len(holidays).rename(columns={'year_month_count': 'holiday_count'})

Let's now generate the previous month holidays count.
We can get that by increasing the month by one (if the holiday count of February was 1 and the holiday count of March was 2, the holiday count of March will be 1).

In [ ]:
prev_holiday_count = holiday_count.copy()
prev_holiday_count.loc[:, 'date'] = prev_holiday_count.loc[:, 'date'] + pd.DateOffset(months=1)
prev_holiday_count = prev_holiday_count.rename(columns={'holiday_count': 'prev_holiday_count'})

Likewise, we can find the next month holiday count by subtracting the months by 1

In [ ]:
next_holiday_count = holiday_count.copy()
next_holiday_count.loc[:, 'date'] = next_holiday_count.loc[:, 'date'] + pd.DateOffset(months=-1)
next_holiday_count = next_holiday_count.rename(columns={'holiday_count': 'next_holiday_count'})

We drop the `date` and create `year` and `month` features we can merge on. 

**NOTE**: In order to merge the date data smoothly afterwards, we should drop the resulting duplicates

In [ ]:
holiday_count.loc[:, 'year'] = holiday_count.loc[:, 'date'].dt.year
holiday_count.loc[:, 'month'] = holiday_count.loc[:, 'date'].dt.month
holiday_count.drop(['date'], axis=1, inplace=True)
holiday_count.drop_duplicates(inplace=True)

In [ ]:
prev_holiday_count.loc[:, 'year'] = prev_holiday_count.loc[:, 'date'].dt.year
prev_holiday_count.loc[:, 'month'] = prev_holiday_count.loc[:, 'date'].dt.month
prev_holiday_count.drop(['date'], axis=1, inplace=True)
prev_holiday_count.drop_duplicates(inplace=True)

In [ ]:
next_holiday_count.loc[:, 'year'] = next_holiday_count.loc[:, 'date'].dt.year
next_holiday_count.loc[:, 'month'] = next_holiday_count.loc[:, 'date'].dt.month
next_holiday_count.drop(['date'], axis=1, inplace=True)
next_holiday_count.drop_duplicates(inplace=True)

We merge the previous, current and next holiday count into one frame.
The resulting `NaN`s will be locations without vacations.
We start by merging with `dates`, as this contains all relevant `year`-`month` combinations

In [ ]:
holidays = pd.merge(dates.loc[:, ['year', 'month']].drop_duplicates(),
                    holiday_count, how='left', on=['year', 'month']).fillna(0)
holidays = pd.merge(holidays, prev_holiday_count, how='left', on=['year', 'month']).fillna(0)
holidays = pd.merge(holidays, next_holiday_count, how='left', on=['year', 'month']).fillna(0)

# Re-shuffle the columns for better overview
holidays = holidays.loc[:, ['year', 'month', 'prev_holiday_count', 'holiday_count', 'next_holiday_count']]

# All columns can be integers
holidays = holidays.astype(np.int32)

# Sort by year and month for better overview
holidays.sort_values(['year', 'month'], inplace=True)
holidays.reset_index(inplace=True, drop=True)

# Add the date block number
holidays.loc[:, 'date_block_num'] = range(holidays.shape[0])

Inspect that we did the correct thing

In [ ]:
holidays

This looks correct. Let's merge these to a common data frame.

In [ ]:
dates = pd.merge(dates, holidays, how='left', on='date_block_num')

In [ ]:
del holiday_count
del next_holiday_count
del prev_holiday_count
del holiday_2012
del holiday_2013
del holiday_2014
del holiday_2015
del holidays
gc.collect()

# Generating aggregated features

## Adding shop features

In [ ]:
shop_drop_cols = ['item_price', 'item_cnt_day', 'item_category_id', 'revenue']
shop = pd.merge(sales_train, items.loc[:, ['item_id', 'item_category_id']], how='left', on=['item_id'])

In [ ]:
shop.loc[:, 'revenue'] = shop.loc[:, 'item_price'] * shop.loc[:, 'item_cnt_day']

### Sum aggregates

In [ ]:
shop_sum = shop.copy()

In [ ]:
shop_sum.loc[:, 'month_shop_revenue_sum'] = \
    shop_sum.loc[:, ['date_block_num', 'shop_id', 'revenue']].\
    groupby(['date_block_num', 'shop_id'])['revenue'].transform(sum)

In [ ]:
shop_sum.loc[:, 'month_shop_item_id_item_cnt_sum'] = \
    shop_sum.loc[:, ['date_block_num', 'shop_id', 'item_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].transform(sum)

In [ ]:
shop_sum.loc[:, 'month_shop_item_cnt_sum'] = \
    shop_sum.loc[:, ['date_block_num', 'shop_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'shop_id'])['item_cnt_day'].transform(sum)

In [ ]:
shop_sum.drop(shop_drop_cols, axis=1, inplace=True)
shop_sum.head()

### Mean aggregates

In [ ]:
shop_mean = shop.copy()

In [ ]:
shop_mean.loc[:, 'shop_revenue_avg'] = \
    shop_mean.loc[:, ['shop_id', 'revenue']].\
    groupby(['shop_id'])['revenue'].transform(np.mean)

In [ ]:
shop_mean.loc[:, 'month_shop_item_cnt_avg'] = \
    shop_mean.loc[:, ['date_block_num', 'shop_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'shop_id'])['item_cnt_day'].transform(np.mean)

In [ ]:
shop_mean.loc[:, 'month_shop_item_id_item_cnt_avg'] = \
    shop_mean.loc[:, ['date_block_num', 'shop_id', 'item_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].transform(np.mean)

In [ ]:
shop_mean.loc[:, 'month_shop_item_cat_item_cnt_avg'] = \
    shop_mean.loc[:, ['date_block_num', 'shop_id', 'item_category_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'shop_id', 'item_category_id'])['item_cnt_day'].transform(np.mean)

In [ ]:
shop_mean.drop(shop_drop_cols, axis=1, inplace=True)
shop_mean.head()

### Other aggregates

In [ ]:
shop_other = shop.copy()

In [ ]:
# NOTE: We use a slightly different notation on this feature as it can be merged on shop instead on shop-item
shop_other.loc[:, 'shop_unique_items_len'] = \
    shop_other.loc[:, ['shop_id', 'item_id']].\
    groupby('shop_id')['item_id'].transform(lambda item_id_group: len(np.unique(item_id_group)))

In [ ]:
shop_other.loc[:, 'month_shop_item_id_item_cnt_max'] = \
    shop_other.loc[:, ['date_block_num', 'shop_id', 'item_id', 'item_cnt_day']].\
    groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].transform(max)

In [ ]:
shop_other.loc[:, 'month_shop_item_id_item_cnt_min'] = \
    shop_other.loc[:, ['date_block_num', 'shop_id', 'item_id', 'item_cnt_day']].\
    groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].transform(min)

In [ ]:
shop_other.drop(shop_drop_cols, axis=1, inplace=True)
shop_other.head()

## Adding non-shop features

In [ ]:
non_shop_drop_cols = ['item_price', 'item_category_id', 'item_cnt_day']
non_shop = pd.merge(sales_train, items.loc[:, ['item_id', 'item_category_id']], how='left', on=['item_id'])

Aggregates by item and month

In [ ]:
non_shop.loc[:, 'month_item_id_item_cnt_sum'] = \
    non_shop.loc[:, ['date_block_num', 'item_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'item_id'])['item_cnt_day'].transform(sum)

In [ ]:
non_shop.loc[:, 'month_item_id_item_cnt_avg'] = \
    non_shop.loc[:, ['date_block_num', 'item_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'item_id'])['item_cnt_day'].transform(np.mean)

In [ ]:
non_shop.loc[:, 'month_item_cat_item_cnt_avg'] = \
    non_shop.loc[:, ['date_block_num', 'item_category_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'item_category_id'])['item_cnt_day'].transform(np.mean)

In [ ]:
non_shop.loc[:, 'month_item_cnt_avg'] = \
    non_shop.loc[:, ['date_block_num', 'item_cnt_day']].\
    groupby(['date_block_num'])['item_cnt_day'].transform(np.mean)

In [ ]:
non_shop.loc[:, 'month_item_id_item_cnt_max'] = \
    non_shop.loc[:, ['date_block_num', 'item_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'item_id'])['item_cnt_day'].transform(max)

In [ ]:
non_shop.loc[:, 'month_item_id_item_cnt_min'] = \
    non_shop.loc[:, ['date_block_num', 'item_id', 'item_cnt_day']].\
    groupby(['date_block_num', 'item_id'])['item_cnt_day'].transform(min)

In [ ]:
non_shop.drop(non_shop_drop_cols, axis=1, inplace=True)
non_shop.head()

#### Clipping the target value

In this comptetition the range of the predicted item count sold should be in the range $[0, 20]$.
This leaves us with two choices:

1. Clipping before training
2. Clipping after prediction

The disatvantage of 1. is that this will give us inconsistencies with other features like the revenue.
However, features like the aggregated revenue of a shop is expected to influence the sales of a shop, so it is not that critical that we reduce the correlation between these features.

On the other hand, if we go for option 2., the range of values which the model is trying to learn from is broad. This can lead to low performance as the target space becomes broad and therefore sparse.

Because of this we clip prior to training.

In [ ]:
shop_mean.loc[:, 'month_shop_item_id_item_cnt_avg'].clip(0, 20, inplace=True)

#### Merge and clean-up

In [ ]:
shop_sum.drop_duplicates(inplace=True)
shop_mean.drop_duplicates(inplace=True)
shop_other.drop_duplicates(inplace=True)
non_shop.drop_duplicates(inplace=True)

In [ ]:
merge_on = ['date', 'date_block_num', 'shop_id', 'item_id']

aggregates = pd.merge(shop_sum, shop_mean, how='left', on=merge_on)
aggregates = pd.merge(aggregates, shop_other, how='left', on=merge_on)
aggregates = pd.merge(aggregates, non_shop, how='left', on=merge_on)

We check that we didn't introduce any `NaN`s or that we accidentally expanded the set 

In [ ]:
if aggregates.isnull().any().any():
    raise AssertionError('NaNs were created')
    
n_aggregates = aggregates.shape[0]
if n_aggregates > n_train_samples:
    raise AssertionError(f'The training set was expanded: '
                         f'n_aggregates={n_aggregates} and n_train_samples={n_train_samples}')

We have no longer use of the day information of the aggregates, so we remove this and remove the duplicates.

In [ ]:
aggregates.drop('date', axis=1, inplace=True)
aggregates.drop_duplicates(inplace=True)
aggregates.reset_index(inplace=True, drop=True)

In [ ]:
print(f'Shape = {aggregates.shape}')
aggregates.head()

We rejoin `item_category_id` as we will use this as a categorical feature

In [ ]:
aggregates = pd.merge(aggregates, items.loc[:, ['item_id', 'item_category_id']], how='left', on='item_id')
print(f'Shape = {aggregates.shape}')

In [ ]:
del shop
del shop_sum
del shop_mean
del shop_other
del non_shop
gc.collect()

# Aggregate EDA

We have now added several aggregated features.
In this section, we are in particular interested in exploring the relation between the categorical features and the target variable.

This exploration will be the fundation to create mean encodings, where the purpose is to code the categorical features (with a lot of features) in such a way that the relation with the target variable is taken into account. 

In [ ]:
target = 'month_shop_item_id_item_cnt_sum'

In [ ]:
shop_item_id_item_cnt_pivot = \
    aggregates.pivot_table(index='shop_id',
                           columns='item_id',
                           values=target,
                           aggfunc='count',
                           fill_value=0)
fig, ax = plt.subplots()
sns.heatmap(shop_item_id_item_cnt_pivot,
            ax=ax,
            cbar=True,
            cmap='viridis', 
            cbar_kws={'label': target})
plt.tight_layout()

We can see that certain items (for example around id $3429$ and $4279$) are sold broadly across all shops, that some shops have quite broad selection (between shop id $24$ and $33$).

We see that with the current encoding, the shops are not clustred around certain items, but are scattered around.

In [ ]:
shop_item_cat_item_cnt_pivot = \
    aggregates.pivot_table(index='shop_id',
                           columns='item_category_id',
                           values=target,
                           aggfunc='count',
                           fill_value=0)
fig, ax = plt.subplots()
sns.heatmap(shop_item_cat_item_cnt_pivot,
            ax=ax,
            cbar=True,
            cmap='viridis', 
            cbar_kws={'label': target})
plt.tight_layout()

We observe that a few categories are dominating the sales (like category $40$ and $56$), and the others are contributing less.

In [ ]:
item_id_item_cat_item_cnt_pivot = \
    aggregates.pivot_table(index='item_id',
                           columns='item_category_id',
                           values=target,
                           aggfunc='count',
                           fill_value=0)
fig, ax = plt.subplots()
sns.heatmap(item_id_item_cat_item_cnt_pivot,
            ax=ax,
            cbar=True,
            cmap='viridis', 
            cbar_kws={'label': target})
plt.tight_layout()

It does not appear that the `item_id` is sorted in terms of `item_category_id`, hence will both provide information to the data set.

In [ ]:
shop_date_block_num_item_cnt_pivot = \
    aggregates.pivot_table(index='shop_id',
                           columns='date_block_num',
                           values=target,
                           aggfunc='count',
                           fill_value=0)
fig, ax = plt.subplots()
sns.heatmap(shop_date_block_num_item_cnt_pivot,
            ax=ax,
            cbar=True,
            cmap='viridis', 
            cbar_kws={'label': target})
plt.tight_layout()

As with the EDA of the raw features, we see that we have seasonal trends, and that a few shops are dominating in terms of quantity.

## Conclusion

From the investigation above, we can argue that it would make sense to create mean encodings for all the categorical features, as they all appear to bring new information to the table.

# Adding mean encoding

We will now mean encode the categorical features above based on how often (on average) the target variable appears in the categorical feature

In [ ]:
cat_features = ['item_id', 'shop_id', 'item_category_id']

In [ ]:
kf = sklearn.model_selection.KFold(n_splits=5, shuffle=False)
new_features = []

# Shortly told we will here aggregate a mean on the training set, 
# store it in the validation set and fill the missing values with the global mean
for train_indices, valid_indices in kf.split(aggregates):
    
    # Train/validation split
    train = aggregates.iloc[train_indices]
    valid = aggregates.iloc[valid_indices]
    
    # Mean encoding
    for feature in cat_features:
        # NOTE: The lines below are equivalent to
        # agg_mean = train.loc[:, [feature, target]].groupby(feature)[target].mean()
        # mean_merged_on_valid_feature = 
        #     pd.merge(valid.loc[:, [feature]], agg_mean.to_frame(), how='left', on=feature).loc[:, feature]
        mean_merged_on_valid_feature = valid.loc[:, feature].map(train.groupby(feature)[target].mean())
    
        # Store the results in aggregates
        aggregates.loc[valid_indices, feature + '_mean_enc'] = mean_merged_on_valid_feature
        
global_mean = aggregates.loc[:, target].mean()
aggregates.fillna(global_mean, inplace=True)

In [ ]:
aggregates.head()

Let's investigate how correlated the mean encodings are

In [ ]:
mean_enc_cols = [col for col in aggregates.columns if 'mean_enc' in col]
corr_coef = aggregates.loc[:,[target, *mean_enc_cols]].corr()

In [ ]:
corr_coef.loc[mean_enc_cols, target].to_frame()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(aggregates.loc[:, 'item_id_mean_enc'], aggregates.loc[:, target], label='item_id', alpha=0.2)
ax.scatter(aggregates.loc[:, 'shop_id_mean_enc'], aggregates.loc[:, target], label='shop_id', alpha=0.2)
ax.scatter(aggregates.loc[:, 'item_category_id_mean_enc'], aggregates.loc[:, target], label='item_category_id', alpha=0.2)
ax.set_ylabel('Target')
ax.set_xlabel('Encoding')
ax.legend(loc='best', fancybox=True, framealpha=0.5)
plt.tight_layout()

We see that the mean encodings are relatively well correlated with the target, and it doesn't appear that there is too much leakage.

Notice, however that the categorical features may be multivalued with respect to the mean encoding (due to the k-fold validation).
This becomes a problem when we will merge the encoded values to the test set.
To solve this, we will aggregate a mean of the mean encoding and merge this with the data data frame.

In [ ]:
aggregates.loc[:, 'item_id_mean_mean_enc'] = \
    aggregates.loc[:, ['item_id_mean_enc', 'item_id']].\
    groupby(['item_id'])['item_id_mean_enc'].transform(np.mean)

aggregates.loc[:, 'shop_id_mean_mean_enc'] = \
    aggregates.loc[:, ['shop_id_mean_enc', 'shop_id']].\
    groupby(['shop_id'])['shop_id_mean_enc'].transform(np.mean)

aggregates.loc[:, 'item_category_id_mean_mean_enc'] = \
    aggregates.loc[:, ['item_category_id_mean_enc', 'item_category_id']].\
    groupby(['item_category_id'])['item_category_id_mean_enc'].transform(np.mean)

In [ ]:
mean_mean_enc_cols = [col for col in aggregates.columns if 'mean_mean_enc' in col]
corr_coef = aggregates.loc[:,[target, *mean_mean_enc_cols]].corr()

In [ ]:
corr_coef.loc[mean_mean_enc_cols, target].to_frame()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(aggregates.loc[:, 'item_id_mean_mean_enc'], aggregates.loc[:, target], label='item_id', alpha=0.2)
ax.scatter(aggregates.loc[:, 'shop_id_mean_mean_enc'], aggregates.loc[:, target], label='shop_id', alpha=0.2)
ax.scatter(aggregates.loc[:, 'item_category_id_mean_mean_enc'], aggregates.loc[:, target], label='item_category_id', alpha=0.2)
ax.set_ylabel('Target')
ax.set_xlabel('Encoding')
ax.legend(loc='best', fancybox=True, framealpha=0.5)
plt.tight_layout()

We see that the main features of the main encoding is preserved.
We therefore remove the original mean encoded features from the aggregated data frame.

In [ ]:
aggregates.drop(['item_id_mean_enc', 'shop_id_mean_enc', 'item_category_id_mean_enc'], axis=1, inplace=True)

# Merging the aggregated features with the data dataframe

Before we make the temporal features, we start by mergning the aggregated features with the data dataframe in order to easily add the lagged features to the test set (month $34$ in the data data frame).

**NOTE**: This section is long, and could probably be improved quite a bit. Essentially what happens is that we merge the different part of the aggregated data frame on the correct columns of the data data frame.

In [ ]:
n_data_samples = data.shape[0]

We start by merging the item category id with the data data frame as we will later use item category as our merge-on column

In [ ]:
merge_on = ['item_id']
data_aggregate = pd.merge(data,
                          items.loc[:, merge_on + ['item_category_id']],
                          how='left', 
                          on=merge_on)

Note that the different columns have different feature dependencies. We must ensure that we are merging the columns on the correct features:

In [ ]:
current_cols = set(aggregates.columns)

mean_enc_cols = [col for col in current_cols if 'mean_mean_enc' in col]

current_cols -= set(mean_enc_cols)

In [ ]:
month_shop_item_id_cols = [col for col in current_cols if 
                           'month' in col and
                           'shop' in col and
                           'item_id' in col]

current_cols -= set(month_shop_item_id_cols)

In [ ]:
month_shop_item_cat_cols = [col for col in current_cols if 
                            'month' in col and
                            'shop' in col and
                            'item_cat' in col]

current_cols -= set(month_shop_item_cat_cols)

In [ ]:
month_shop_cols = [col for col in current_cols if 
                   'month' in col and
                   'shop' in col]

current_cols -= set(month_shop_cols)

In [ ]:
month_item_id_cols = [col for col in current_cols if 
                      'month' in col and
                      'item_id' in col]

current_cols -= set(month_item_id_cols)

In [ ]:
month_item_cat_cols = [col for col in current_cols if 
                       'month' in col and
                       'item_cat' in col]

current_cols -= set(month_item_cat_cols)

In [ ]:
month_cols = [col for col in current_cols if 
              'month' in col]

current_cols -= set(month_cols)

In [ ]:
shop_cols = [col for col in current_cols if 
            'shop' in col and
             col != 'shop_id']

identifier_cols = current_cols - set(shop_cols)

Check that we did the correct thing

In [ ]:
if set(identifier_cols) != {'date_block_num', 'item_category_id', 'item_id', 'shop_id'}:
    raise AssertionError('identifier_cols is not correct')

Merge

In [ ]:
merge_on = ['date_block_num', 'shop_id', 'item_id']
data_aggregate = pd.merge(data_aggregate,
                          aggregates.loc[:, merge_on + month_shop_item_id_cols].drop_duplicates(),
                          how='left', 
                          on=merge_on)

As we have expanded our dataset with shop id - item id combinations that have not been sold during that month, we can safely replace the `NaN`s prior to week $34$ to $0$.

In [ ]:
# NOTE: The inplace operator seem not to work (maybe because we are selecting a slice)
#       Hence we use the assign operator
data_aggregate.loc[data_aggregate.loc[:, 'date_block_num']<34, month_shop_item_id_cols] =\
    data_aggregate.loc[data_aggregate.loc[:, 'date_block_num']<34, month_shop_item_id_cols].fillna(0)

In [ ]:
merge_on = ['date_block_num', 'shop_id', 'item_category_id']
data_aggregate = pd.merge(data_aggregate,
                          aggregates.loc[:, merge_on + month_shop_item_cat_cols].drop_duplicates(),
                          how='left', 
                          on=merge_on)

Due to the shop id - item id expansion, new combinations of shop id - item category id may appear. 
We can alson safely replace the `NaN`s prior to week $34$ to $0$.

In [ ]:
# NOTE: The inplace operator seem not to work (maybe because we are selecting a slice)
#       Hence we use the assign operator
data_aggregate.loc[data_aggregate.loc[:, 'date_block_num']<34, month_shop_item_cat_cols] =\
    data_aggregate.loc[data_aggregate.loc[:, 'date_block_num']<34, month_shop_item_cat_cols].fillna(0)

In [ ]:
merge_on = ['date_block_num', 'shop_id']
data_aggregate = pd.merge(data_aggregate, 
                          aggregates.loc[:, merge_on + month_shop_cols].drop_duplicates(), 
                          how='left',
                          on=merge_on)

No new `NaN`s should appear after the above merge as the expansion of the training set does not introduce new shops.

In [ ]:
merge_on = ['date_block_num', 'item_id']
data_aggregate = pd.merge(data_aggregate,
                          aggregates.loc[:, merge_on + month_item_id_cols].drop_duplicates(),
                          how='left',
                          on=merge_on)

No new `NaN`s should appear after the above merge as the expansion of the training set does not introduce new items.

In [ ]:
merge_on = ['date_block_num', 'item_category_id']
data_aggregate = pd.merge(data_aggregate,
                          aggregates.loc[:, merge_on + month_item_cat_cols].drop_duplicates(),
                          how='left',
                          on=merge_on)

No new `NaN`s should appear after the above merge as the expansion of the training set does not introduce new item categories.

In [ ]:
merge_on = ['date_block_num']
data_aggregate = pd.merge(data_aggregate,
                          aggregates.loc[:, merge_on + month_cols].drop_duplicates(), 
                          how='left',
                          on=merge_on)

No new `NaN`s should appear after the above merge as the expansion of the training set does not introduce new months.

In [ ]:
merge_on = ['shop_id']
data_aggregate = pd.merge(data_aggregate,
                          aggregates.loc[:, merge_on + shop_cols].drop_duplicates(),
                          how='left', 
                          on=merge_on)

No new `NaN`s should appear after the above merge as the expansion of the training set does not introduce new shops.

Finally, we merge the mean encoded features:

In [ ]:
merge_on = ['shop_id']
data_aggregate = pd.merge(data_aggregate, 
                          aggregates.loc[:, merge_on + ['shop_id_mean_mean_enc']].drop_duplicates(),
                          how='left',
                          on=merge_on)

No new `NaN`s should appear after the above merge as the expansion of the training set does not introduce new shops.

In [ ]:
merge_on = ['item_id']
data_aggregate = pd.merge(data_aggregate, 
                          aggregates.loc[:, merge_on + ['item_id_mean_mean_enc']].drop_duplicates(),
                          how='left',
                          on=merge_on)

No new `NaN`s should appear after the above merge as the expansion of the training set does not introduce new items.

In [ ]:
merge_on = ['item_category_id']
data_aggregate = pd.merge(data_aggregate, 
                          aggregates.loc[:, merge_on + ['item_category_id_mean_mean_enc']].drop_duplicates(),
                          how='left',
                          on=merge_on)

No new `NaN`s should appear after the above merge as the expansion of the training set does not introduce new categories.

We check that we didn't introduce any `NaN`s in the training set or that we accidentally expanded the set 

In [ ]:
if data_aggregate.loc[data_aggregate.loc[:, 'date_block_num']<34].isnull().any().any():
    raise AssertionError('NaNs were created')
    
n_data_aggregate = data_aggregate.shape[0]
if n_aggregates > n_data_samples:
    raise AssertionError(f'The set was expanded: '
                         f'n_aggregates={n_aggregates} and n_data_aggregate={n_data_aggregate}')

Only two type of features should now contain `NaN`s in the test set:

* Monthly features - as these are based on item counts not present in the test set
* Item id features - as the test set contains new items

In [ ]:
test_nan_features = data_aggregate.loc[data_aggregate.loc[:, 'date_block_num']==34].isnull().any()
print(test_nan_features)

We check that the monthly features contains only `NaN`s

In [ ]:
cols_w_month = [col for col in data_aggregate.columns if 'month' in col]
if not data_aggregate.loc[data_aggregate.loc[:, 'date_block_num']==34, cols_w_month].isnull().all().all():
    raise AssertionError('The monthly features contained something else than NaNs')   

We need to fill the `NaN` values for the non-month features

In [ ]:
nan_col = [col for col, is_nan in zip(test_nan_features.index, test_nan_features.values) if is_nan]
non_month_nan_cols = [col for col in data_aggregate.columns if col not in cols_w_month and col in nan_col]
print(non_month_nan_cols)

There are several options to fill these values.
As we saw from the EDA of the raw data, only $1.6 %$ of the item ids were present in the test set that was not present in the training set. 
Thus, it should not matter too much what we fill these values with. 
We choose to use the mean encoding trick and fill them with the global mean.

In [ ]:
data_aggregate.loc[:, 'item_id_mean_mean_enc'].fillna(global_mean, inplace=True)

# Adding temporal history

As this is a sequential problem, we would like to incorperate some time information into the training set.

Note that all the montly aggregated features we generated above will only be present as lagged features in the test set when we create the lagged features.

In [ ]:
def make_lagged(df, col, lags, merge_on, fillna=0):
    """
    Makes lagged features
    
    We make the lag this by adding the lag number to date_block_num
    and merge the result on date_block_num of the corresponding month of the input df.
    
    Parameters
    ----------
    df : DataFrame
        The feature to make lagged features from
    col : str
        The name of the feature
    lags : list
        The number of months to lag
    merge_on : list
        The columns to merge on
    fillna : float
        The value to fill the NaNs with
    
    Returns
    -------
    lag_df : DataFrame
        A DataFrame containing the lagged features
    """
    
    lag_df = df.loc[:, merge_on + [col]].copy()
    
    samples = lag_df.shape[0]
    
    for lag in lags:
        print(f'Processing lag {lag}', end='\r')
        tmp_df = lag_df.copy()
        tmp_df.loc[:, 'date_block_num'] = tmp_df.loc[:, 'date_block_num'] + lag
        new_col = f'{col}_lag_{lag}'
        tmp_df.rename({col: new_col}, axis=1, inplace=True)
        tmp_df.drop_duplicates(inplace=True)
        lag_df = pd.merge(lag_df, tmp_df.loc[:, [*merge_on, new_col]], how='left', on=merge_on)
    
    lag_df.fillna(fillna, inplace=True)
    
    # Drop the original feature
    lag_df.drop(col, axis=1, inplace=True)
    
    cur_samples = lag_df.shape[0]
    if cur_samples > samples:
        raise AssertionError(f'Sample size increased. Old: {samples}, new: {cur_samples}')
    
    return lag_df

Our strategy is to lag the features in the following way:

**Long lag** (i.e. by $1$, $2$, $3$, $6$, $9$ and $12$ months):
These are the features which time trend is belived to be most important for prediction
* Merge on: month, shop, item
    * `month_shop_item_id_item_cnt_sum`
* Merge on: month, shop
    * `month_shop_item_cnt_sum`
* Merge on: month, item
    * `month_item_id_item_cnt_sum`

**Short lag** (i.e. by only $1$ month)
These are the features which time trend is belived to be less important for prediction
* Merge on: month, shop, item
    * `month_shop_item_id_item_cnt_max`
    * `month_shop_item_id_item_cnt_min`
    * `month_shop_item_id_item_cnt_avg`
* Merge on: month, shop, category
    * `month_shop_item_cat_item_cnt_avg`
* Merge on: month, shop
    * `month_shop_item_cnt_avg`
    * `month_shop_revenue_sum`
* Merge on: month, item
    * `month_item_id_item_cnt_max`
    * `month_item_id_item_cnt_min`
    * `month_item_id_item_cnt_avg`
* Merge on: month, category
    * `month_item_cat_item_cnt_avg`
* Merge on: month
    * `month_item_cnt_avg`

**Fillna strategy**:

For the shop-item combinations we can safely fill the resulting `NaN`s with $0$ as the combination simply did not sell anything the previous `lag`-months.
In fact, the same argument goes for all other mergin strategies as well.

## Long time lag

In [ ]:
long_lags = [1, 2, 3, 6, 9, 12]

In [ ]:
merge_on = ['date_block_num', 'item_id', 'shop_id']
tmp = make_lagged(df=data_aggregate,
                  col='month_shop_item_id_item_cnt_sum',
                  lags=long_lags,
                  merge_on=merge_on)

We make a minor sanity check: We select the highest number of `'month_shop_item_id_item_cnt_sum'` which is in a month where the lag is visible in the last lag feature, and see if the number is constant for the lag months

In [ ]:
col = 'month_shop_item_id_item_cnt_sum'
rows = data_aggregate.loc[(data_aggregate.loc[:, col]==\
                           data_aggregate.loc[data_aggregate.loc[:, 'date_block_num']<33-max(long_lags), col].max())]

month = rows.loc[:, 'date_block_num'].values[0]
shop_id = rows.loc[:, 'shop_id'].values[0]
item_id = rows.loc[:, 'item_id'].values[0]
val = rows.loc[:, col].values[0]

for lag in long_lags:
    lagged_val = tmp.loc[(tmp.loc[:, 'date_block_num'] == month + lag) & 
                   (tmp.loc[:, 'shop_id'] == shop_id) &
                   (tmp.loc[:, 'item_id'] == item_id),
                   merge_on + [f'{col}_lag_{lag}']].drop_duplicates().\
                 loc[:, f'{col}_lag_{lag}']
    if not np.isclose(val, lagged_val):
        raise AssertionError('Oh dear, something is wrong with the lag function...')

print('Lag function looks OK!')

In [ ]:
data_aggregate = pd.merge(data_aggregate, tmp.drop_duplicates(), how='left', on=merge_on)

In [ ]:
def add_lagged_feature(df, col, merge_on, lags):
    """
    Add a feature to the input data frame
    
    Parameters
    ----------
    df : DataFrame
        The data frame to add the lagged features to
    col : str
        The feature to add make the lagged feature of
    merge_on : list
        List of features to merge on
    lags : list
        The lags

    Returns
    -------
    df : DataFrame
        The data frame with added features
    """
    tmp = make_lagged(df=df,
                      col=col,
                      lags=lags,
                      merge_on=merge_on)
    df = pd.merge(df, tmp.drop_duplicates(), how='left', on=merge_on)
    
    return df

In [ ]:
merge_on = ['date_block_num', 'shop_id']
data_aggregate = add_lagged_feature(data_aggregate, 'month_shop_item_cnt_sum', merge_on, long_lags)

In [ ]:
merge_on = ['date_block_num', 'item_id']
data_aggregate = add_lagged_feature(data_aggregate, 'month_item_id_item_cnt_sum', merge_on, long_lags)

## Short time lag

In [ ]:
short_lags = [1]

In [ ]:
merge_on = ['date_block_num', 'shop_id', 'item_id']
data_aggregate = add_lagged_feature(data_aggregate, 'month_shop_item_id_item_cnt_max', merge_on, short_lags)
data_aggregate = add_lagged_feature(data_aggregate, 'month_shop_item_id_item_cnt_min', merge_on, short_lags)
data_aggregate = add_lagged_feature(data_aggregate, 'month_shop_item_id_item_cnt_avg', merge_on, short_lags)

In [ ]:
merge_on = ['date_block_num', 'shop_id', 'item_category_id']
data_aggregate = add_lagged_feature(data_aggregate, 'month_shop_item_cat_item_cnt_avg', merge_on, short_lags)

In [ ]:
merge_on = ['date_block_num', 'shop_id']
data_aggregate = add_lagged_feature(data_aggregate, 'month_shop_item_cnt_avg', merge_on, short_lags)
data_aggregate = add_lagged_feature(data_aggregate, 'month_shop_revenue_sum', merge_on, short_lags)

In [ ]:
merge_on = ['date_block_num', 'item_id']
data_aggregate = add_lagged_feature(data_aggregate, 'month_item_id_item_cnt_max', merge_on, short_lags)
data_aggregate = add_lagged_feature(data_aggregate, 'month_item_id_item_cnt_min', merge_on, short_lags)
data_aggregate = add_lagged_feature(data_aggregate, 'month_item_id_item_cnt_avg', merge_on, short_lags)

In [ ]:
merge_on = ['date_block_num', 'item_category_id']
data_aggregate = add_lagged_feature(data_aggregate, 'month_item_cat_item_cnt_avg', merge_on, short_lags)

In [ ]:
merge_on = ['date_block_num']
data_aggregate = add_lagged_feature(data_aggregate, 'month_item_cnt_avg', merge_on, short_lags)

Verfiy that we did not introduced `NaN`s after the last 

In [ ]:
cols_w_lag = [col for col in data_aggregate.columns if '_lag_' in col]
if data_aggregate.loc[data_aggregate.loc[:, 'date_block_num'] > max(long_lags), cols_w_lag].isnull().any().any():
    raise AssertionError('The monthly features contained something else than NaNs')
    
if data_aggregate.shape[0] > n_data_samples:
    raise AssertionError(f'The set was expanded: '
                         f'n_aggregates={n_aggregates} and n_data_aggregate={n_data_aggregate}')

In [ ]:
del tmp
gc.collect()

# Adding leakage features

The leakage features are features where we use information about the test set.

As both shop id and item id are features of the test set, and since these are not related to time, these are leakages.

### Number of ids in train and test

In [ ]:
shop_id_train = sales_train.loc[:, 'shop_id']
shop_id_test = sales_test.loc[:, 'shop_id']
shop_id_both = pd.concat([shop_id_train, shop_id_test], axis=0).to_frame()
shop_id_both.loc[:, 'shop_id_count'] = shop_id_both.groupby('shop_id')['shop_id'].transform(len)

# NOTE: Drop duplicated as we want to merge
shop_id_both.drop_duplicates(inplace=True)

In [ ]:
item_id_train = sales_train.loc[:, 'item_id']
item_id_test = sales_test.loc[:, 'item_id']
item_id_both = pd.concat([item_id_train, item_id_test], axis=0).to_frame()
item_id_both.loc[:, 'item_id_count'] = item_id_both.groupby('item_id')['item_id'].transform(len)

# NOTE: Drop duplicated as we want to merge
item_id_both.drop_duplicates(inplace=True)

Out of curiosity we check how these are distributed

In [ ]:
fig, ax = plt.subplots()
shop_id_both.loc[:, 'shop_id_count'].hist(ax=ax, bins=200)
ax.set_xlabel('shop_id_count')
ax.set_ylabel('count')
plt.tight_layout()

In [ ]:
shop_id_both.loc[:, 'shop_id_count'].value_counts().describe()

It appears that the number of rows for each `shop_id` is well spread, and not clustering around a specific number

In [ ]:
fig, ax = plt.subplots()
item_id_both.loc[:, 'item_id_count'].hist(ax=ax, bins=200)
ax.set_xlabel('item_id_count')
ax.set_ylabel('count')
plt.tight_layout()

In [ ]:
item_id_both.loc[:, 'item_id_count'].value_counts().describe()

In [ ]:
item_id_both.head()

### The ID

As we saw from the EDA, we saw that the `ID` was highly correlated to the `shop_id`, so we include it here. Item and shops without an ID will be given `-1` (although we could probably construct a more appropriate `ID` feature if we checked the feature more)

In [ ]:
id_df = pd.merge(sales_train, sales_test, how='left', on=['shop_id', 'item_id'])
id_df.loc[:,'ID'].fillna(-1, inplace=True)
id_df.loc[:,'ID'] = id_df.loc[:,'ID'].astype('int32')

### Row number

As the test set contains data after the train data, these will have a higher row number.
Of course, we could be unlucky and have a test set which is shuffled with respect to the training set, but we nevertheless give it a shot

In [ ]:
row_train = pd.DataFrame(list(range(len(sales_train.index))), columns=['row_nr'])
row_test = pd.DataFrame(np.array(range(len(sales_test.index)))+row_train.iloc[-1].values[0], columns=['row_nr'])

# Adding text features

Taking into the possibility that the names are correlated to the target, we add some text features as well. We split `item_name`, `shop_name` and `item_category_name` into cyrillic and latin words. We will stem these, and then combine them again before fitting a TF-IDF model to them.

**NOTE**: The TF-IDF model does not care about the relative position of the words, so it is ok if the order is scrambled when recombining the words to sentences again.

We would now like to stem the words (ideally we would like to lemmatize the words, but it looks like the lemmatization for non-english languages are not as readily available at the moment).

**NOTE**: The stemmer casts to lowercase

In [ ]:
russian_stemmer = nltk.stem.SnowballStemmer('russian')
english_stemmer = nltk.stem.SnowballStemmer('english')

In [ ]:
def separate_cyrillic_latin(words):
    """
    Separates the cyrillic and latin words
    
    Notes
    -----
    This function does not conserve word order
    
    Parameters
    -----------
    words : str
        The string of words to be split
        
    Returns
    -------
    separated_words : str
        The words separated by _SEP_
        Cyrillic words are to the left of the separator, the latin to the right
    """
    
    words_split = words.split(' ')
    cyrillic_words = list()
    latin_words = list()
    
    for word in words_split:
        # https://stackoverflow.com/questions/48255244/python-check-if-a-string-contains-cyrillic-characters
        if re.search('[а-яА-Я]', word) is not None:
            cyrillic_words.append(word)
        else:
            latin_words.append(word)
    
    cyrillic_words = ' '.join(cyrillic_words)
    latin_words = ' '.join(latin_words)
    
    separated_words = f'{cyrillic_words}_SEP_{latin_words}'
    
    return separated_words

In [ ]:
def get_text_features(df, col, return_all=False):
    """
    Returns a new DataFrame with added text features
    
    Parameters
    -----------
    df : DataFrame
        The data frame to add the text features to
    col : str
        The column to obtain the text features from
    return_all : bool
        If True, intermediate columns will be returned
        
    Returns
    -------
    df_nlp : DataFrame
        The data frame with the added text features
        * {col}_clean - col column cleaned so that only alphabetical and numerical characters are present 
                        (only returned if return_all is True)
        * cyrillic_latin - column where cyrillic and latin letters has been separated 
                           (only returned if return_all is True)
        * cyrillic - column with only stemmed cyrillic words present (only returned if return_all is True)
        * latin - column with only stemmed latin words present (only returned if return_all is True)
        * {col}_nlp - combination of the cyrillic and latin column described above
        * {col}_cyrillic_words - cyrillic word count
        * {col}_latin_words - latin word count
        * {col}_total_words - total word count
    """
    
    df_nlp = df.copy()
    
    # First we clean the text by removing non-alphabetical characters and non-numeric characters
    
    df_nlp.loc[:, f'{col}_clean'] = \
    df_nlp.loc[:, f'{col}'].apply(lambda s: re.sub('[^а-яА-Яa-zA-Z0-9 ]', ' ', s))

    # Remove duplicated whitespaces
    df_nlp.loc[:, f'{col}_clean'] = \
        df_nlp.loc[:, f'{col}_clean'].apply(lambda s: re.sub(' +',' ', s))
    
    df_nlp.loc[:, 'cyrillic_latin'] = df_nlp.loc[:, f'{col}_clean'].apply(separate_cyrillic_latin)
    df_nlp.loc[:, 'cyrillic'] = df_nlp.loc[:, 'cyrillic_latin'].apply(lambda s: s.split('_SEP_')[0])
    df_nlp.loc[:, 'latin'] = df_nlp.loc[:, 'cyrillic_latin'].apply(lambda s: s.split('_SEP_')[1])
    
    df_nlp.loc[:, 'cyrillic'] = df_nlp.loc[:, 'cyrillic'].apply(russian_stemmer.stem)
    df_nlp.loc[:, 'latin'] = df_nlp.loc[:, 'latin'].apply(english_stemmer.stem)
    
    # Recombine words
    df_nlp.loc[:, f'{col}_nlp'] = df_nlp.loc[:, 'cyrillic'].str[:] + ' ' + df_nlp.loc[:, 'latin'].str[:]
    
    # We add the word count of each type together with the total.
    # The rationale for doing is
    # 1. It's possible that product with complex names are not sold as much
    # 2. In case there is a lot of English words in the product, it could be that it's less sellable in Russia
    # 3. Possible other reasons not mentioned here
    
    df_nlp.loc[:, f'{col}_cyrillic_words'] = \
        df_nlp.loc[:, 'cyrillic'].apply(lambda s: len(s.split(' ')) if s != '' else 0)
    df_nlp.loc[:, f'{col}_latin_words'] = \
        df_nlp.loc[:, 'latin'].apply(lambda s: len(s.split(' ')) if s != '' else 0)
    
    # NOTE: This is in fact an interaction feature
    df_nlp.loc[:, f'{col}_total_words'] = \
        df_nlp.loc[:, f'{col}_cyrillic_words'] + df_nlp.loc[:, f'{col}_latin_words']
    
    if not return_all:
        remove = [f'{col}_clean', 'cyrillic_latin', 'cyrillic', 'latin']
        df_nlp.drop(remove, axis=1, inplace=True)
    
    return df_nlp

In [ ]:
item_nlp = get_text_features(items, 'item_name')
item_category_nlp = get_text_features(item_categories, 'item_category_name')
shop_nlp = get_text_features(shops, 'shop_name')

Check how many tokens we are dealing with

In [ ]:
item_corpus = ' '.join(item_nlp.loc[:, 'item_name_nlp'].values)
item_corpus_tokens = nltk.word_tokenize(item_corpus)
print(f'Unique item_name_tokens {len(set(item_corpus_tokens))}')

In [ ]:
item_category_corpus = ' '.join(item_category_nlp.loc[:, 'item_category_name_nlp'].values)
item_category_corpus_tokens = nltk.word_tokenize(item_category_corpus)
print(f'Unique item_category_name_tokens {len(set(item_category_corpus_tokens))}')

In [ ]:
shop_corpus = ' '.join(shop_nlp.loc[:, 'shop_name_nlp'].values)
shop_corpus_tokens = nltk.word_tokenize(shop_corpus)
print(f'Unique shop_name_tokens {len(set(shop_corpus_tokens))}')

We should take care not to use all tokens as this may result in a [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality). 
Let's see how the words are distributed

In [ ]:
samples = 30

In [ ]:
plt.figure()
fd_item = nltk.FreqDist(item_corpus_tokens)
fd_item.plot(samples, cumulative=False)
plt.tight_layout()

In [ ]:
plt.figure()
fd_item_category = nltk.FreqDist(item_category_corpus_tokens)
fd_item_category.plot(samples, cumulative=False)
plt.tight_layout()

In [ ]:
plt.figure()
fd_shop = nltk.FreqDist(shop_corpus_tokens)
fd_shop.plot(samples, cumulative=False)
plt.tight_layout()

We can see that a couple of words constitutes the most of the corpuses. In other words we can expect a high information gain from the first couple of features and diminishing returns as we add more words. We will from graphical inspection try with max features $35$ for TF-IDF for the item corpus, $25$ for the item category corpus and $10$ for the shop corpus. 

In [ ]:
item_features = 35
item_category_features = 25
shop_features = 10

In [ ]:
tf_idf_item_vec = sklearn.feature_extraction.text.TfidfVectorizer(max_features=item_features)
tf_idf_item = tf_idf_item_vec.fit_transform(item_nlp['item_name_nlp']).toarray()

In [ ]:
tf_idf_item_category_vec = sklearn.feature_extraction.text.TfidfVectorizer(max_features=item_category_features)
tf_idf_item_category = tf_idf_item_category_vec.fit_transform(item_category_nlp['item_category_name_nlp']).toarray()

In [ ]:
tf_idf_shop_vec = sklearn.feature_extraction.text.TfidfVectorizer(max_features=shop_features)
tf_idf_shop = tf_idf_shop_vec.fit_transform(shop_nlp['shop_name_nlp']).toarray()

Combine the TF-IDF results with the corresponding data frames

In [ ]:
col_names = [f'item_tf_idf_{i}' for i in range(tf_idf_item.shape[1])]
tf_idf_item_df = pd.DataFrame(tf_idf_item, columns=col_names)
item_nlp = pd.concat([item_nlp, tf_idf_item_df], axis=1)
item_nlp.drop(['item_name', 'item_category_id', 'item_name_nlp'], axis=1, inplace=True)

In [ ]:
col_names = [f'item_category_tf_idf_{i}' for i in range(tf_idf_item_category.shape[1])]
tf_idf_item_category_df = pd.DataFrame(tf_idf_item_category, columns=col_names)
item_category_nlp = pd.concat([item_category_nlp, tf_idf_item_category_df], axis=1)
item_category_nlp.drop(['item_category_name', 'item_category_name_nlp'], axis=1, inplace=True)

In [ ]:
col_names = [f'shop_tf_idf_{i}' for i in range(tf_idf_shop.shape[1])]
tf_idf_shop_df = pd.DataFrame(tf_idf_shop, columns=col_names)
shop_nlp = pd.concat([shop_nlp, tf_idf_shop_df], axis=1)
shop_nlp.drop(['shop_name', 'shop_name_nlp'], axis=1, inplace=True)

# Type casting?

Only makes sense for tree methods

# Feature dropping

Let's now drop the features which are no longer needed.
We will only keep those who are needed for the training.

**NOTE**: Although tempting we should not get rid of `item_id` and `shop_id` even though we got `ID`, as these are used to identify the objects under investigation. Also note that these features are not ordinal, and it is possible that some very clever label encoding for these exists.

In [ ]:
drop_cols = ['date', 
             'item_category_id', 
             'item_category_name', 
             'item_cnt_day', 
             'item_name',
             'item_price',
             'revenue',
             'shop_name']
merged_train.drop(drop_cols, axis=1, inplace=True)

In [ ]:
drop_cols = [col for col in drop_cols if col in merged_test.columns]
merged_test.drop(drop_cols, axis=1, inplace=True)

**NOTE**: There is no need to [normalize the target](https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re), however, we will use the target as a temporal feature (see below), which means that the temporal feature needs to be normalized.

# Merging

In [ ]:
merged_train = pd.concat([merged_train, row_train], axis=1)
merged_test = pd.concat([merged_test, row_test], axis=1)

We see that most item ids are present only a couple of times. This means that we have little amount of item level information for most items.

In [ ]:
merged_train = pd.merge(merged_train, item_id_both, how='left', on=['item_id'])
merged_train = pd.merge(merged_train, shop_id_both, how='left', on=['shop_id'])

merged_test = pd.merge(merged_test, item_id_both, how='left', on=['item_id'])
merged_test = pd.merge(merged_test, shop_id_both, how='left', on=['shop_id'])

Merge with the train set and the test set

In [ ]:
merged_train = pd.merge(merged_train, item_nlp, how='left', on=['item_id'])
merged_train = pd.merge(merged_train, item_category_nlp, how='left', on=['item_category_id'])
merged_train = pd.merge(merged_train, shop_nlp, how='left', on=['shop_id'])

In [ ]:
merged_test = pd.merge(merged_test, item_nlp, how='left', on=['item_id'])
merged_test = pd.merge(merged_test, item_category_nlp, how='left', on=['item_category_id'])
merged_test = pd.merge(merged_test, shop_nlp, how='left', on=['shop_id'])

In [ ]:
merged_train.loc[:, 'target_month'] = merged_train.loc[:, 'target'].copy()
target = merged_train.loc[:, ['ID', 'item_id', 'shop_id', 'date_block_num', 'target']].copy()
merged_train.drop('target', axis=1, inplace=True)

**NOTE**: Left-joining the data frames with the sales on the left conserves the training rows

In [ ]:
merged_train = pd.merge(sales_train, items, how='left', on=['item_id'])
merged_train = pd.merge(merged_train, item_categories, how='left', on=['item_category_id'])
merged_train = pd.merge(merged_train, shops, how='left', on=['shop_id'])

**NOTE**: As we are making several `group_by` statements when generating the features, we should be careful and not combine the train and test set prior to the generation. Instead we should merge the generated features when possible, if not we should separately generate these features for the test set.

In [ ]:
merged_test = pd.merge(sales_test, items, how='left', on=['item_id'])
merged_test = pd.merge(merged_test, item_categories, how='left', on=['item_category_id'])
merged_test = pd.merge(merged_test, shops, how='left', on=['shop_id'])

In [ ]:
merged_train.head()

In [ ]:
merged_test.head()

Note that we already have the following encoding:

- `shop_id` - `shop_name`
- `item_id` - `item_name`
- `item_category_id` - `item_category_name`

Merge `shop_n_products` to `merged_test`

In [ ]:
shop_products = merged_train.loc[:, ['shop_n_products', 'shop_id', 'item_id']]
shop_products.drop_duplicates(inplace=True)
merged_test = pd.merge(merged_test, shop_products,
                       how='left', on=['shop_id', 'item_id'])

In case a shop-item doens't exist in the train set, but in the test set we will set it to $-1$ as we do not know its value

In [ ]:
merged_test.loc[:, 'shop_n_products'].fillna(-1, inplace=True)

# Normalization

We recall that tree-based models does not depend on the normalization, but non-tree-based models hugely depend on them. As we plan to use ensemble methods for the predictions we should normalize our data. Let's go through the normalization strategy for each of the features:

We will use MaxMinScaler on these ordinal features
* `ID`
* `date_block_num`
* `item_id` 
* `month`
* `quarter`
* `row_nr`
* `shop_id`
* `year`

We will use StandardScaler on these numerical features in order to keep the distribution
* `days_in_month`
* `holiday_count`
* `item_category_name_cyrillic_words`
* `item_category_name_latin_words`
* `item_category_name_total_words`
* `item_id_count`
* `item_id_mean_enc_target_month`
* `item_count_high_month`
* `item_count_low_month`
* `item_count_mean_month`
* `item_count_sum_month`
* `item_name_cyrillic_words`
* `item_name_latin_words`
* `item_name_total_words`
* `item_price`
* `next_holiday_count`
* `prev_holiday_count`
* `revenue`
* `shop_id_count`
* `shop_item_cnt_month`
* `shop_item_count_high_month`
* `shop_item_count_low_month`
* `shop_item_count_mean_month`
* `shop_n_products`
* `shop_name_cyrillic_words`
* `shop_name_latin_words`
* `shop_name_total_words`
* `shop_revenue_month`
* `target_month`

These are already normalized
* `item_category_tf_idf_*`
* `item_tf_idf_*`
* `shop_tf_idf_*`

Due to time constraints, we will not use the Rank scaler or recast distribution (so that the become more Gaussian) with functions like `np.log`, although this could improve the quality of prediction from for example neural nets.

We make copies of `ID`, `date_block_num`, `item_id` and `shop_id` so that we can operate with one scaled and one unscaled version

In [ ]:
copies = ['ID', 'date_block_num', 'item_id', 'shop_id']

for copy in copies:
    merged_train.loc[:, f'{copy}_scaled'] = merged_train.loc[:, f'{copy}'].copy()
    merged_test.loc[:, f'{copy}_scaled'] = merged_train.loc[:, f'{copy}'].copy()

**NOTE**: We postpone normalization of `date_block_num`, `item_id` and `shop_id` as we will use these to merge temporal features on later

In [ ]:
features_max_min = [
    'ID_scaled',
    'date_block_num_scaled',
    'item_id_scaled',
    'month',
    'quarter',
    'row_nr',
    'shop_id_scaled',
    'year']

max_min_scaler = sklearn.preprocessing.MinMaxScaler()

max_min_scaler.fit(merged_train.loc[:, features_max_min])
merged_train.loc[:, features_max_min] = max_min_scaler.transform(merged_train.loc[:, features_max_min])

**NOTE**: We use the same scaler for train and test

In [ ]:
features_max_min = [col for col in features_max_min if col in merged_test]
merged_test.loc[:, features_max_min] = max_min_scaler.transform(merged_test.loc[:, features_max_min])

In [ ]:
features_standard_scaler = [
    'days_in_month',
    'holiday_count',
    'item_category_name_cyrillic_words',
    'item_category_name_latin_words',
    'item_category_name_total_words',
    'item_id_count',
    'item_id_mean_enc_target_month',
    'item_count_high_month',
    'item_count_low_month',
    'item_count_mean_month',
    'item_count_sum_month',
    'item_name_cyrillic_words',
    'item_name_latin_words',
    'item_name_total_words',
    'next_holiday_count',
    'prev_holiday_count',
    'shop_id_count',
    'shop_item_cnt_month',
    'shop_item_count_high_month',
    'shop_item_count_low_month',
    'shop_item_count_mean_month',
    'shop_n_products',
    'shop_name_cyrillic_words',
    'shop_name_latin_words',
    'shop_name_total_words',
    'shop_revenue_month',
    'target_month'
    ]

standard_scaler = sklearn.preprocessing.StandardScaler()

test_features_standard_scaler = \
    [col for col in features_standard_scaler if col in merged_test]
train_features_standard_scaler = set(features_standard_scaler) - set(test_features_standard_scaler)

# Exlusive train features
standard_scaler.fit(merged_train.loc[:, train_features_standard_scaler])
merged_train.loc[:, train_features_standard_scaler] = \
    standard_scaler.transform(merged_train.loc[:, train_features_standard_scaler])

# Train and test features
standard_scaler.fit(merged_train.loc[:, test_features_standard_scaler])
merged_train.loc[:, test_features_standard_scaler] = \
    standard_scaler.transform(merged_train.loc[:, test_features_standard_scaler])

In [ ]:
merged_test.loc[:, test_features_standard_scaler] = \
    standard_scaler.transform(merged_test.loc[:, test_features_standard_scaler])

# Checking correlation

Out of curiosity we check how the correlation map looks

In [ ]:
fig, ax = plt.subplots()
corr = merged_train.corr()
cax = ax.matshow(corr)
fig.colorbar(cax)

## Fill in the blanks

We note that the merging process has created a lot of NaNs

In [ ]:
check_col = 'shop_revenue_month_lag_1'
isnull = merged_train.loc[:, f'{check_col}'].isnull()

isnull_pct = 100*merged_train.loc[isnull, f'{check_col}'].shape[0]/merged_train.shape[0]

print(f'{isnull_pct:.2f} % of the rows of {check_col} in the training set bcontains NaNs')

Although there may exist smarter ways to fill the NaN values (with the mean, median or a reconstructed values), we will simply fill them with $0$.

In [ ]:
merged_train.fillna(0, inplace=True)
merged_test.fillna(0, inplace=True)

**NOTE**: With this method we need to throw away `n_lag` months of our training set

# Storing the data frames

Guard

In [ ]:
diff = set(merged_train.columns).symmetric_difference(set(merged_test.columns))

if len(diff) != 0:
    raise AssertionError(f'Difference in columns of merged_train and merged_test found:\n{diff}')
    
if n_train_samples != merged_train.shape[0]:
    raise AssertionError(f'Train samples introduced:\nn_train_samples={n_train_samples}, '
                         f'merged_train.shape[0]={merged_train.shape[0]}')

if n_test_samples != merged_test.shape[0]:
    raise AssertionError(f'Test samples introduced:\nn_train_samples={n_test_samples}, '
                         f'merged_test.shape[0]={merged_test.shape[0]}')

In [ ]:
generated_data = Path('.').absolute().joinpath('generated_data')
generated_data.mkdir(exist_ok=True)

merged_train.to_hdf(generated_data.joinpath('train.hdf'), key='train')
target.to_hdf(generated_data.joinpath('target.hdf'), key='target')
merged_test.to_hdf(generated_data.joinpath('test.hdf'), key='test')